Данная задача чем-то похожа на CBIR, если говорить понятным языком, то на поиск в Яндексе по картинке других похожих изображений, или текста связанного с изображением. Подобные задачи решаются с помощью представления исходного изображения в виде вектора в некотором пространстве меньшей размерности, так называемый embedding. Затем эмбеддинги сравниваются и находят ближайшие к нему, это и будут похожие изображения.
В нашем случае изображения кодируются с помощью CLIP от OpenAI, а затем полученные эмбеддинги кластеризуются с помощью AgglomerativeClustering из sklearn. Можно попробовать другие методы кодирования и кластеризации, они тоже работают. (но не все, старушка VGG-19 у меня не сработала, то ли вектора взял большие 4000 признаков, то ли в целом сеть старовата и не смогла)

In [1]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, download_root='E:\clip')

E:\anaconda3\envs\with_torch_cuda\lib\site-packages\torch\serialization.py:799: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, download_root='E:\clip')

import os

images = [i for i in os.listdir('dataset')]
from tqdm import tqdm
features_2 = []
for i in tqdm(range(len(images))):
    path = images[i]
    image = preprocess(Image.open('dataset/'+path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    features_2.append(image_features.detach().cpu().numpy())

from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components=300)
X = pca.fit_transform(np.array(features_2).squeeze())


from sklearn.cluster import AgglomerativeClustering

clf = AgglomerativeClustering(n_clusters=983)
clf.fit(X)

import pandas as pd

res = pd.DataFrame(columns=['filename','label'])
res['filename'] = images
res['label'] = clf.labels_
res.to_csv('submit.csv',index=None)

100%|██████████| 9605/9605 [02:00<00:00, 79.74it/s] 
